In [63]:
import pandas as pd
import numpy as numpy

import re
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

import itertools
import requests

import gender_guesser.detector as gender

import warnings
warnings.filterwarnings('ignore')

#### Borial Management

In [43]:
url = "https://borialmanagement.com/"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("h4", {"class": "personName toBold"})

In [44]:
actores = []
for tag in tags:
    actores.append(tag.text)

In [45]:
actores.remove('GONZALO FERNГЃNDEZ')
actores.remove('VГЌCTOR CASTILLO')
actores.remove('ГЃLVARO QUINTANA')
actores.remove('CARLA DГЌAZ')
actores.remove('MARIO GARCГЌA')
actores.remove('SANTI MARГЌN')
actores.remove('ГЃLVARO FONTALBA')

In [46]:
actores2 = []
for actor in actores:
    if len(re.findall("\W", actor)) > 1:
        pass
    else:
        actores2.append(actor)

In [47]:
actores3 = []
for actor in actores2:
    actores3.append(actor.split())

In [48]:
d = gender.Detector()

nombres_por_genero = {"masculino": [], "femenino": []}

for nombre in actores3:
    nombre_completo = nombre[0].capitalize() + nombre[1].capitalize()
    genero = d.get_gender(nombre[0].capitalize())
    if genero == "male":
        nombres_por_genero["masculino"].append(nombre_completo)
    elif genero == "female":
        nombres_por_genero["femenino"].append(nombre_completo)

In [49]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

dicc = {"actor": [], "formacion": []}

for k, v in nombres_por_genero.items():
    if k == "masculino":
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actores/{nombres_por_genero['masculino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            if resultado[1] == "FORMACIÓN":
                artista = resultado[0].lower()
                resultado.pop(0)
                resultado.pop(0)
                i = 0
                while resultado[i] != "TELEVISIÓN":
                    dicc["formacion"].append(resultado[i])
                    dicc["actor"].append(artista)
                    i+=1
            else:
                dicc["formacion"].append("no hay formación")
                dicc["actor"].append(artista)
    else:
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actrices/{nombres_por_genero['femenino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            if resultado[1] == "FORMACIÓN":
                artista = resultado[0].lower()
                resultado.pop(0)
                resultado.pop(0)
                i = 0
                print(len(resultado))
                while (resultado[i] != "TELEVISIÓN") and (resultado[i] != "TEATRO") and (resultado[i] != "CINE"):
                    print(i)
                    dicc["formacion"].append(resultado[i])
                    dicc["actor"].append(artista)
                    i+=1
            else:
                dicc["formacion"].append("no hay formación")
                dicc["actor"].append(artista)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.49)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D6DCE3+50899]
	(No symbol) [0x00CFE111]
	(No symbol) [0x00C05588]
	(No symbol) [0x00BED333]
	(No symbol) [0x00C4F4DB]
	(No symbol) [0x00C5DB33]
	(No symbol) [0x00C4B6F6]
	(No symbol) [0x00C27708]
	(No symbol) [0x00C2886D]
	GetHandleVerifier [0x00FD3EAE+2566302]
	GetHandleVerifier [0x010092B1+2784417]
	GetHandleVerifier [0x0100327C+2759788]
	GetHandleVerifier [0x00E05740+672048]
	(No symbol) [0x00D08872]
	(No symbol) [0x00D041C8]
	(No symbol) [0x00D042AB]
	(No symbol) [0x00CF71B7]
	BaseThreadInitThunk [0x75F37D69+25]
	RtlInitializeExceptionChain [0x7707B74B+107]
	RtlClearBits [0x7707B6CF+191]


In [ ]:
df_borial = pd.DataFrame(dicc)
df_borial.head(2)

,actor,formacion
0,daniel albaladejo,Licenciado en Interpretación por la E.S.A.D. d...
1,daniel albaladejo,Cursos de perfeccionamiento con:


#### Única Representaciones

In [ ]:
url = "https://www.unicarepresentaciones.com/Actores"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [ ]:
h_unica = []
for tag in tags:
    h_unica.append(tag.text)

In [ ]:
url = "https://www.unicarepresentaciones.com/Actrices"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [ ]:
m_unica = []
for tag in tags:
    m_unica.append(tag.text)

In [ ]:
actores_unica = h_unica+m_unica

In [ ]:
h_unica_scrap = []
for actor in (h_unica):
    h_unica_scrap.append(actor.strip().replace(" ", "-").lower())

In [ ]:
m_unica_scrap = []
for actriz in (m_unica):
    m_unica_scrap.append(actriz.strip().replace(" ", "-").lower())

In [ ]:
unica_scrap = h_unica_scrap+m_unica_scrap

In [ ]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

dicc = {"actor": [], "formacion": []}

for actor in unica_scrap:
    try:
        url = f"https://www.unicarepresentaciones.com/actores-actrices/{actor}"
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        driver.implicitly_wait(10)
        cv = driver.find_element("css selector", "#block-system-main > div > div.ficha-actores > div.ficha-col-1").text
        elementos = cv.split("\n")
        elementos.pop(0)
        elementos = [i.strip() for i in elementos]
        if "Formación" not in elementos:
            pass
        else:
            indice_formacion = elementos.index("Formación") + 1
            lista_formacion = elementos[indice_formacion:]

            dicc["actor"].append(actor)
            dicc["formacion"].append(lista_formacion)
    except NoSuchElementException:
        driver.close()

KeyboardInterrupt: 

In [ ]:
formacion_unica = pd.DataFrame(dicc)

In [ ]:
formacion_unica = formacion_unica.explode('formacion')

#### Listado IMDB

In [ ]:
listado_actores = actores2 + actores_unica
listado_actores[:2]

['DANIEL ALBALADEJO', 'ENRIQUE ASENJO']

In [64]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

lista = []

for actor in listado_actores[:2]:
    url = "https://www.imdb.com/?ref_=nv_home"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.implicitly_wait(10)
    driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.TAB)
    driver.find_element("css selector", "#suggestion-search-button").click()
    sleep(1)
    
    element = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-4f91839f-0.hmcKDt > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.ffAEHI > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a')))
    element.click()
    
    lista.append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div:nth-child(5) > div > div > div.ipc-accordion__item__content > div").text)


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (371, 565)
  (Session info: chrome=112.0.5615.49)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D6DCE3+50899]
	(No symbol) [0x00CFE111]
	(No symbol) [0x00C05588]
	(No symbol) [0x00C35A7A]
	(No symbol) [0x00C34336]
	(No symbol) [0x00C3267B]
	(No symbol) [0x00C31797]
	(No symbol) [0x00C294A5]
	(No symbol) [0x00C4B8FC]
	(No symbol) [0x00C28EC6]
	(No symbol) [0x00C4BC54]
	(No symbol) [0x00C5E01C]
	(No symbol) [0x00C4B6F6]
	(No symbol) [0x00C27708]
	(No symbol) [0x00C2886D]
	GetHandleVerifier [0x00FD3EAE+2566302]
	GetHandleVerifier [0x010092B1+2784417]
	GetHandleVerifier [0x0100327C+2759788]
	GetHandleVerifier [0x00E05740+672048]
	(No symbol) [0x00D08872]
	(No symbol) [0x00D041C8]
	(No symbol) [0x00D042AB]
	(No symbol) [0x00CF71B7]
	BaseThreadInitThunk [0x75F37D69+25]
	RtlInitializeExceptionChain [0x7707B74B+107]
	RtlClearBits [0x7707B6CF+191]


In [62]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

lista = []

for actor in listado_actores[:2]:
    url = "https://www.imdb.com/?ref_=nv_home"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.implicitly_wait(10)
    driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.TAB)
    driver.find_element("css selector", "#suggestion-search-button").click()
    sleep(1)
    driver.find_element("css selector", '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-4f91839f-0.hmcKDt > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.ffAEHI > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a').click()
    lista.append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div:nth-child(5) > div > div > div.ipc-accordion__item__content > div").text)

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (371, 563)
  (Session info: chrome=112.0.5615.49)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D6DCE3+50899]
	(No symbol) [0x00CFE111]
	(No symbol) [0x00C05588]
	(No symbol) [0x00C35A7A]
	(No symbol) [0x00C34336]
	(No symbol) [0x00C3267B]
	(No symbol) [0x00C31797]
	(No symbol) [0x00C294A5]
	(No symbol) [0x00C4B8FC]
	(No symbol) [0x00C28EC6]
	(No symbol) [0x00C4BC54]
	(No symbol) [0x00C5E01C]
	(No symbol) [0x00C4B6F6]
	(No symbol) [0x00C27708]
	(No symbol) [0x00C2886D]
	GetHandleVerifier [0x00FD3EAE+2566302]
	GetHandleVerifier [0x010092B1+2784417]
	GetHandleVerifier [0x0100327C+2759788]
	GetHandleVerifier [0x00E05740+672048]
	(No symbol) [0x00D08872]
	(No symbol) [0x00D041C8]
	(No symbol) [0x00D042AB]
	(No symbol) [0x00CF71B7]
	BaseThreadInitThunk [0x75F37D69+25]
	RtlInitializeExceptionChain [0x7707B74B+107]
	RtlClearBits [0x7707B6CF+191]
